In [ ]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [ ]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [64]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [19]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [2]:
data = pd.read_csv("matches.csv", header=0, encoding="latin-1")

prepared_data = pd.DataFrame()

homeVenue = list()
for i, d in data["team1Text"].iteritems():
    v = data["venue"].get(i)
    h = False
    if isinstance(v, str) and d in v:
        h = True
    homeVenue.append(h)
    
prepared_data["date"] = data.date
prepared_data["name"] = preprocessing.LabelEncoder().fit_transform(data["team1"])
prepared_data["opponentName"] = preprocessing.LabelEncoder().fit_transform(data["team2"])
prepared_data["homeVenue"] = homeVenue
prepared_data["neutralVenue"] = [not v for v in homeVenue]
prepared_data["homeScore"] = data.team1Score
prepared_data["opponentScore"] = data.team2Score
prepared_data["differenceScore"] = data.team1Score - data.team2Score
prepared_data["homeWin"] = data.team1Score > data.team2Score
prepared_data["opponentWin"] = data.team1Score < data.team2Score
prepared_data["draw"] = data.team1Score == data.team2Score

In [10]:
cols = prepared_data.columns
x_columns = cols.drop(["homeScore", "opponentScore", "differenceScore"])

x = prepared_data[x_columns]
y = prepared_data[["homeScore"]]
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [66]:
clf_lr = LogisticRegression(penalty='l1', tol=0.01, C=0.1)
print(clf_lr.fit(X_train, y_train.values.ravel()).score(X_test, y_test))

X_new = X_test[0:15]
print(X_new[0:1])
pred = clf_lr.predict(X_new)
print(pred)

0.416394208313872
           date  name  opponentName  homeVenue  neutralVenue  homeWin  \
17099  20141118   193           143       True         False     True   

       opponentWin   draw  
17099        False  False  
[2 2 2 2 2 2 2 0 1 1 2 2 0 2 0]
